In [1]:
import pandas as pd
import mysql.connector

In [2]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_43'     
)
cursor = connection.cursor()

### Bài 1

In [3]:
# Bảng Transactions_Raw vi phạm 1NF vì cột stock_codes có chứa danh sách, tập hợp trong 1 cột

### Bài 2

In [4]:
cursor.execute("""
CREATE TABLE Transactions (
    transaction_id INTEGER AUTO_INCREMENT PRIMARY KEY,
    customer_name VARCHAR(200) NOT NULL, 
    transaction_date DATE NOT NULL
);
""")

In [5]:
cursor.execute("""
INSERT INTO Transactions (transaction_id, customer_name, transaction_date) VALUES
(1, 'Nguyen Van A', '2024-12-01'),
(2, 'Le Thi B', '2024-12-02'),                
(3, 'Tran Van C', '2024-12-03');
""")

connection.commit()

In [6]:
cursor.execute("""
CREATE TABLE StockCodes (
    transaction_id INTEGER NOT NULL,
    stock_code VARCHAR(5) NOT NULL, 
    FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id)
);
""")

In [7]:
cursor.execute("""
INSERT INTO StockCodes (transaction_id, stock_code) VALUES
(1, 'VNM'),
(1, 'FPT'),
(1, 'MWG'), 
(2, 'SSI'),                
(3, 'VCB'), 
(3, 'TCB');
""")

connection.commit()

### Bài 3

In [8]:
# Bảng Transactions có các cột: transaction_id (khóa chính), customer_name, transaction_date
# Bảng StockCodes có các cột: transaction_id (khóa ngoại), stock_code

### Bài 4

In [9]:
query = ("""
SELECT a.customer_name, b.stock_code
FROM Transactions a
LEFT JOIN StockCodes b ON b.transaction_id = a.transaction_id
""")

data_4a = pd.read_sql_query(query, connection)
print(data_4a)

  customer_name stock_code
0  Nguyen Van A        VNM
1  Nguyen Van A        FPT
2  Nguyen Van A        MWG
3      Le Thi B        SSI
4    Tran Van C        VCB
5    Tran Van C        TCB


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_23648\1450763689.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4a = pd.read_sql_query(query, connection)


In [10]:
cursor.execute("""
UPDATE Transactions
SET customer_name = 'Nguyen V. A'
WHERE customer_name = 'Nguyen Van A';
""");

connection.commit()

In [11]:
cursor.execute("""
DELETE FROM StockCodes
WHERE stock_code = 'MWG' AND transaction_id IN (
    SELECT transaction_id
    FROM Transactions
    WHERE customer_name = 'Nguyen V. A');
""")

connection.commit()

### Bài 5

In [12]:
query = ("""
SELECT a.customer_name, COUNT(DISTINCT b.stock_code) AS total_stock_codes
FROM Transactions a
LEFT JOIN StockCodes b ON b.transaction_id = a.transaction_id
GROUP BY a.customer_name;
""")

data_5a = pd.read_sql_query(query, connection)
print(data_5a)

  customer_name  total_stock_codes
0      Le Thi B                  1
1   Nguyen V. A                  2
2    Tran Van C                  2


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_23648\3082205704.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5a = pd.read_sql_query(query, connection)


In [15]:
query = ("""
SELECT a.customer_name, COUNT(DISTINCT b.stock_code) AS total_stock_codes
FROM Transactions a
LEFT JOIN StockCodes b ON b.transaction_id = a.transaction_id
GROUP BY a.customer_name
ORDER BY total_stock_codes DESC
LIMIT 1;
""")

data_5b = pd.read_sql_query(query, connection)
print(data_5b)

  customer_name  total_stock_codes
0   Nguyen V. A                  2


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_23648\2014431092.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_5b = pd.read_sql_query(query, connection)
